In [1]:
import tensorflow as tf
import numpy.random as rnd

In [2]:
#Configure the log directory for TensorBoard
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_log = "tmp/logs/"
logs_path = "{}/run-{}/".format(root_log, now)


In [3]:
#CONSTRUCTION PHASE

# model config
n_steps = 28
n_inputs = 28 #28 features, 1 per pixel
n_neurons = 150
n_outputs = 10
n_layers = 3

# set hyperparameters
learning_rate = 0.001

# reset everything to rerun in jupyter
tf.reset_default_graph()

with tf.name_scope('input'):
    #create input and output placeholders
    #X has shape [    t = 0              t = 1          t = 28 (n_steps)
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  instance 1
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  instance 2
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  instance 3
    #             ...
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  total instances in batch (None | Undefined)
    #            ]
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    #Y is an integer 0 - 9
    y = tf.placeholder(tf.int32, [None])

with tf.name_scope('model'):
    #create the cells and the layers of the network
    layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) for layer in range(n_layers)]
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers, state_is_tuple=True)
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

    #For classification, we only want the output activation at the last time step so we use 'outputs' 
    #We transpose so that the time axis is first and use tf.gather() for selecting the last index, x28. 

    #X has shape [    instance 1       instance 2      instance n (150)
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  t = 0
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  t = 1
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  t = 2
    #             ...
    #             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  t = 28
    #            ]
    outputs = tf.transpose(outputs, [1, 0, 2])

    #gather the final values across t = 28
    last = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)

with tf.name_scope('softmax'):
    #Create Softmax layer with XEntropy 
    logits = tf.layers.dense(last, n_outputs)
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)

with tf.name_scope('loss'):
    #Compute the loss, mean xentropy score
    loss = tf.reduce_mean(xentropy)

# specify optimizer
with tf.name_scope('train'):
    #Here we use Adaptive Momement Estimation which keeps track of an exponentially decaying average of past gradients
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

#Compute accuracy
with tf.name_scope('accuracy'):
    #Does the highest logit correspond to the target class?
    #in_top_k returns a 1D tensor full of boolean values
    correct = tf.nn.in_top_k(logits, y, 1)
    #We cast to floating point numbers and then take the mean to compute an accuracy score
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# create a summary for our cost and accuracy
tf.summary.scalar("loss", loss)
tf.summary.scalar("accuracy", accuracy)
        
# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.summary.merge_all()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
#IMPORT MNIST DATASET
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
#TRAINING PHASE
n_epochs = 100
batch_size = 150

#Adds an op to initialize all variables in the model
init = tf.global_variables_initializer()

#create log writer object
writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

#open a TensorFlow Session
with tf.Session() as sess:
    init.run()
        
    #Epoch is a single pass through the entire training set, followed by testing of the verification set.
    for epoch in range(n_epochs):
        #Number of batches, here we exhaust the training set
        for iteration in range(mnist.train.num_examples // batch_size):   

            #obtain batch of specified batch size
            X_batch, y_batch = mnist.train.next_batch(batch_size) 
                        
            #reshape the array to size n_steps * n_inputs
            X_batch = X_batch.reshape((-1, n_steps, n_inputs)) 
            
            #feed in the batch
            #sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
            _, summary = sess.run([training_op, summary_op], feed_dict={X: X_batch, y: y_batch})
        
            #write log
            writer.add_summary(summary, epoch * int(mnist.train.num_examples/batch_size) + iteration)
            
        #compute accuracy of RNN against Training Set and Test Set
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
writer.close()

0 Train accuracy: 0.973333 Test accuracy: 0.9496
1 Train accuracy: 0.946667 Test accuracy: 0.9646
2 Train accuracy: 0.973333 Test accuracy: 0.9711
3 Train accuracy: 0.966667 Test accuracy: 0.977
4 Train accuracy: 0.966667 Test accuracy: 0.9712
5 Train accuracy: 0.966667 Test accuracy: 0.9739
6 Train accuracy: 0.98 Test accuracy: 0.9732
7 Train accuracy: 0.973333 Test accuracy: 0.9714
8 Train accuracy: 0.993333 Test accuracy: 0.9768
9 Train accuracy: 0.993333 Test accuracy: 0.9787
10 Train accuracy: 0.986667 Test accuracy: 0.9765
11 Train accuracy: 0.966667 Test accuracy: 0.9744
12 Train accuracy: 0.98 Test accuracy: 0.9791
13 Train accuracy: 0.986667 Test accuracy: 0.9769
14 Train accuracy: 1.0 Test accuracy: 0.9829
15 Train accuracy: 0.973333 Test accuracy: 0.9762
16 Train accuracy: 0.986667 Test accuracy: 0.9764
17 Train accuracy: 1.0 Test accuracy: 0.9782
18 Train accuracy: 1.0 Test accuracy: 0.9775
19 Train accuracy: 0.993333 Test accuracy: 0.9794
20 Train accuracy: 1.0 Test accura